In [1]:
import pandas as pd
import numpy as np

import requests

from py_ball import league, image, boxscore, player, playerdashptshotdefend

# static lists to get players and teams
from nba_api.stats.static import players
from nba_api.stats.static import teams

# endpoints to get the list of players on every team and shot chart for each player
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import teamplayerdashboard

# getting nba defensive statistics
from nba_api.stats.endpoints import boxscoredefensive
from nba_api.stats.endpoints import defensehub
from nba_api.stats.endpoints import leaguedashptdefend
from nba_api.stats.endpoints import playerdashptshotdefend
from nba_api.stats.endpoints import boxscoretraditionalv2


# getting nba player info for their position
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import playerdashboardbyyearoveryear

HEADERS = {'Connection': 'keep-alive',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'Referer': 'stats.nba.com',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Accept-Language': 'en-US,en;q=0.9',
           "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6)' +\
                         ' AppleWebKit/537.36 (KHTML, like Gecko)' + \
                         ' Chrome/81.0.4044.129 Safari/537.36'}

## Finding all the nba shot for each player from each game for the 2019/20 nba season

In [2]:
# importing a dictionary of the nba teams to get a list of the team ids

teams_dict = teams.get_teams()

In [3]:
# obtaining list of team ideas for each team in the team dictionary

team_ids = [team['id'] for team in teams_dict]

In [ ]:
df_list = []

In [ ]:
# looping through each team id in the list of team ids
for team in team_ids:
    # obtaining the dashboard of the teams with all the active players and their stats
    team_players = teamplayerdashboard.TeamPlayerDashboard(team, season = '2019-20')

    # using that list to grab the player ids from every player on every team
    player_ids = team_players.players_season_totals.get_data_frame().PLAYER_ID

    # looping through all the players on every team
    for player_id in player_ids:

        # getting the shot chart object from the nba api for each player using the team id and the player id
        shot_chart = shotchartdetail.ShotChartDetail(team, player_id)

        # turning that shot chart object into a dataframe
        shot_chart_df = shot_chart.get_data_frames()[0]

        # changing the dates into a datetime to grab the games from 2019-2020, then setting that as the index
        shot_chart_df['GAME_DATE'] = pd.to_datetime(shot_chart_df['GAME_DATE'], format = '%Y%m%d', errors = 'ignore')
        shot_chart_df.set_index('GAME_DATE', inplace = True)

        # locating only games for the 2019/20 nba season
        shot_chart_2020 = shot_chart_df.loc['2019-10-22':'2020-03-11']

        # appending it to a list to later concat into a single DataFrame
        df_list.append(shot_chart_2020)

In [ ]:
# concating the list of dataframes into a single dataframe

df = pd.concat(df_list)

In [ ]:
# storing that in a csv

df.to_csv('2019-20_nba_shot_charts.csv')

## Finding all shot charts for all players from 1999-00 season to present

In [ ]:
# getting a list of all the nba seasons from 1999-00 season to present

seasons = ['2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14',
           '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', 
           '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00']

In [ ]:
# list to append all the dataframes of player shot charts to
all_seasons_list = []

# list to store player ids so the same player isnt used more than once
used_players = []


# looping through each team id in the list of team ids
for team in team_ids:
    
    # looping through each season to grab the players from the team in that season
    for season in seasons:
        
        try:
            # obtaining the dashboard of the teams with all the active players and their stats
            team_players = teamplayerdashboard.TeamPlayerDashboard(team, season = season)

            # using that list to grab the player ids from every player on every team
            player_ids = team_players.players_season_totals.get_data_frame().PLAYER_ID
            
            # looping through all the players on every team
            for player_id in player_ids:
                
                if player_id not in used_players:
                    
                    # getting the shot chart object from nba api for each player using team id and player id
                    shot_chart = shotchartdetail.ShotChartDetail(team, player_id)

                    # turning that shot chart object into a dataframe
                    shot_chart_df = shot_chart.get_data_frames()[0]

                    # changing the dates into a datetime to grab the games from 2019-2020, then setting that as the index
                    shot_chart_df['GAME_DATE'] = pd.to_datetime(shot_chart_df['GAME_DATE'], format = '%Y%m%d', errors = 'ignore')
                    shot_chart_df.set_index('GAME_DATE', inplace = True)

                    # locating only games for the 2019/20 nba season
                    shot_chart_all = shot_chart_df.loc['1999-11-02':'2020-03-11']
                    
                    # appending it to a list to later concat into a single DataFrame
                    all_seasons_list.append(shot_chart_all)
                    
                    # appending the player id to store and not use again in case it comes up for another team
                    used_players.append(player_id)
                    
                else:
                    pass
                
        except JSONDecodeError:
            pass

In [ ]:
all_seasons = pd.concat(all_seasons_list)

In [ ]:
all_seasons.to_csv('2000-2020_shot_charts.csv')

This gave me an issue as it only output the made shots so i used this temporarily for my data but later took just the game ids to use with the py_ball library which gave all the shots i wanted. Will eventually be doing the same for the 2019-20 season when comparing players later on.

## Finding nba defensive statistics

In [ ]:
# # getting a list of all the games that i gathered shot charts for so i can now use the game ids to gather defensive
# # statistics for each player in each game to group them in the same way i did with the shot charts

# player_ids_df = pd.read_csv('2000-2020_shot_charts.csv', index_col = 'GAME_DATE')


In [ ]:
# # getting unique games as to not overlap and get the same game multiple times

# game_ids = player_ids_df.GAME_ID.unique()

In [ ]:
# # getting an empty list to store the dataframes of every box score
# boxscore_list = []

In [ ]:
# # getting all defensive box score stats for each game

# # looping through each unique game
# for game in game_ids:
    
#     # getting the boxscore object from the nba api and converting it into a dataframe
#     boxscore = boxscoredefensive.BoxScoreDefensive(game_id = '00' + str(game))
#     boxscore_df = boxscore.get_data_frames()[0]
    
#     # appending the box score to the empty list to eventually concat into a df
#     boxscore_list.append(boxscore_df)

In [ ]:
# # creating a dataframe of all the defensive box scores

# boxscore_df = pd.concat(boxscore_list)

In [ ]:
# # saving to a csv

# boxscore_df.to_csv('defensive_box_scores.csv')

Kept timing out and wouldn't save all the games as i wanted, also did not have the season the game was played in, just the game id which would have led to more problems going into the future. Will be using Py_ball library to try and get data for this instead that I can then concat with the shot data.

___
# PY_BALL

## Using Py_ball for shot charts

In [5]:
all_seasons = pd.read_csv('2010-2020_shot_charts.csv', index_col = 'GAME_DATE')

In [10]:
unique_games = all_seasons.GAME_ID.unique()

In [23]:
# getting a list of all the nba seasons from 1999-00 season to present

seasons = ['2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14',
           '2012-13', '2011-12', '2010-11']

In [17]:
# list to append all the dataframes of player shot charts to
shots_list = []


league_id = '00' #NBA
player_id = '0' #All players

# looping through each team id in the list of team ids
for game in unique_games:
    
    # looping through each season to grab the players from the team in that season
    for season in seasons:
        
        shots = player.Player(headers = HEADERS, endpoint = 'shotchartdetail', 
                              league_id = league_id, player_id = player_id, 
                              game_id = '00' + str(game), season = season)
        shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
        
        shots_list.append(shot_df)

KeyboardInterrupt: 

## Getting Defensive statistics through py_ball

In [ ]:
# getting a list of unique players to loop through and get their defensive stats

unique_players = all_seasons.PLAYER_ID.unique()

In [ ]:
# looping through all the different players to get the dataframe of their defensive shot 
# location metrics and store the dataframe to a list (this only works for the newer players)

defensive_shooting = []

league_id = '00'

for ids in unique_players:
    defense = player.Player(headers = HEADERS, endpoint = 'playerdashptshotdefend', 
                           league_id = league_id, player_id = ids)
    defensive_shooting.append(pd.DataFrame(defense.data['DefendingShots']))


In [ ]:
defensive_shots_df = pd.concat(defensive_shooting)

defensive_shots_df.to_csv('defensive_shooting_by_player.csv')